In [1]:
import pgmpy as pg
from pgmpy.readwrite import BIFReader
from pgmpy.base import UndirectedGraph
import itertools
import networkx as nx

In [2]:
data = BIFReader("./data/alarm.bif")

In [26]:

# return all the set of vertices adjacent to a or b in graph G not including a and b. 
def get_AG(G, a, b):
    neighborsA = list(G.neighbors(a))     
    neighborsB = list(G.neighbors(b))
    unique = list(set(neighborsA + neighborsB))
    if a in unique:
        unique.remove(a)
    if b in unique:
        unique.remove(b)
    return unique

def I(G, a, S, b):
    print(list(S))
    tmpG = G.copy()
    for node in S:
        tmpG.remove_node(node)
    shortestPath = nx.shortest_path(tmpG, source=a, target=b)
    print("Shortest Path between ", a, " and ", b, "-> ", shortestPath)
    return len(shortestPath) == 0

In [27]:
# Inizializing variables

Z = data.get_values().keys()
pi = data.get_parents()

G1 = UndirectedGraph()
G1.add_nodes_from(Z)
edges = itertools.combinations(Z, 2)
edges = list(edges)
G1.add_edges_from(edges)
# order is ord because ord is a keyword in python
order = 0
old_pi = {}
for node in Z:
    old_pi[node] = []
old_prob = 0
u = 15

tmp_edges = edges[:]
for (a,b) in tmp_edges:
    # unique nodes adjacet to a or b
    unique = get_AG(G1, a, b)
    if len(unique) >= order:
        for subset in itertools.combinations(unique, order):
            if I(G1, a, subset, b):
                G1.remove_edge(a, b)
    # forse non va qui
    else:
        order = order + 1
print(G1)

[]
Shortest Path between  HISTORY  and  CVP ->  ['HISTORY', 'CVP']
[]
Shortest Path between  HISTORY  and  PCWP ->  ['HISTORY', 'PCWP']
[]
Shortest Path between  HISTORY  and  HYPOVOLEMIA ->  ['HISTORY', 'HYPOVOLEMIA']
[]
Shortest Path between  HISTORY  and  LVEDVOLUME ->  ['HISTORY', 'LVEDVOLUME']
[]
Shortest Path between  HISTORY  and  LVFAILURE ->  ['HISTORY', 'LVFAILURE']
[]
Shortest Path between  HISTORY  and  STROKEVOLUME ->  ['HISTORY', 'STROKEVOLUME']
[]
Shortest Path between  HISTORY  and  ERRLOWOUTPUT ->  ['HISTORY', 'ERRLOWOUTPUT']
[]
Shortest Path between  HISTORY  and  HRBP ->  ['HISTORY', 'HRBP']
[]
Shortest Path between  HISTORY  and  HREKG ->  ['HISTORY', 'HREKG']
[]
Shortest Path between  HISTORY  and  ERRCAUTER ->  ['HISTORY', 'ERRCAUTER']
[]
Shortest Path between  HISTORY  and  HRSAT ->  ['HISTORY', 'HRSAT']
[]
Shortest Path between  HISTORY  and  INSUFFANESTH ->  ['HISTORY', 'INSUFFANESTH']
[]
Shortest Path between  HISTORY  and  ANAPHYLAXIS ->  ['HISTORY', 'ANAPHYL

In [ ]:
def SGS(a, b, ord):
    return 0